# Motif neuron set examples

In [2]:
import obi_one as obi
from pathlib import Path

### __Initialization:__ Loading a circuit *with* connectivity matrix

In [4]:
circuit_name = "N_10__top_nodes_dim6"

circuit_path_prefix = Path("../data/tiny_circuits")
matrix_path_prefix = Path("../data/connectivity_matrices")  # OPTIONAL: Connectivity matrix path only required for some of the node sets in this example notebook; can be set to None

circuit_path = circuit_path_prefix / circuit_name / "circuit_config.json"
if matrix_path_prefix is None:
    matrix_path = None
else:
    matrix_path = matrix_path_prefix / circuit_name / "connectivity_matrix.h5"
circuit = obi.Circuit(name=circuit_name, path=str(circuit_path), matrix_path=str(matrix_path))

print(f"Circuit '{circuit}' with {circuit.sonata_circuit.nodes[circuit.default_population_name].size} neurons and {circuit.sonata_circuit.edges[circuit.default_edge_population_name].size} synapses")
print(f"Default node population: '{circuit.default_population_name}'")
print(f"Default edge population: '{circuit.default_edge_population_name}'")

Circuit 'N_10__top_nodes_dim6' with 10 neurons and 176 synapses
Default node population: 'S1nonbarrel_neurons'
Default edge population: 'S1nonbarrel_neurons__S1nonbarrel_neurons__chemical'


### __Example 1:__ Neuron set with all reciprocal E-E neuron pairs within a layer

In [5]:
neuron1_filter = {"synapse_class": "EXC", "layer": "6"}  # First neuron A in pair
neuron2_filter = {"synapse_class": "EXC", "layer": "6"}  # Second neuron B in pair

conn_ff_filter = {"nsyn": {"gt": 0}}  # Feedforward connectivity from A->B
conn_fb_filter = {"nsyn": {"gt": 0}}  # Feedback connectivity from B->A

pair_selection = {}  # Select all pairs

neuron_set = obi.PairMotifNeuronSet(neuron1_filter=neuron1_filter, neuron2_filter=neuron2_filter, conn_ff_filter=conn_ff_filter, conn_fb_filter=conn_fb_filter, pair_selection=pair_selection)


In [6]:
# List of selected pairs
neuron_set.get_pair_table(circuit, circuit.default_population_name)

,nrn1,nrn2,nsyn_ff,nsyn_fb,nsyn_all,is_rc
0,3,1,6,5,11,True
1,1,3,5,6,11,True
2,9,4,16,3,19,True
3,4,9,3,16,19,True


In [7]:
# Neuron set expression
neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

PairMotifNeuronSet resolved in population 'S1nonbarrel_neurons' of circuit 'N_10__top_nodes_dim6':
> Neuron IDs (4): [1 3 4 9]
> Node set dict: {'population': 'S1nonbarrel_neurons', 'node_id': [1, 3, 4, 9]}


### __Example 2:__ Strongest connected E-E neuron pair within a layer (but which is not reciprocal)

In [8]:
neuron1_filter = {"node_set": "Excitatory", "layer": "6"}
neuron2_filter = {"node_set": "Excitatory",  "layer": "6"}

conn_ff_filter = {"nsyn": {"gt": 0}}
conn_fb_filter = {"nsyn": 0}  # No feedback connection

pair_selection = {"count": 1, "method": "max_nsyn_ff"}  # Selection based on max. number of synapses

neuron_set = obi.PairMotifNeuronSet(neuron1_filter=neuron1_filter, neuron2_filter=neuron2_filter, conn_ff_filter=conn_ff_filter, conn_fb_filter=conn_fb_filter, pair_selection=pair_selection)


In [9]:
# List of selected pairs
neuron_set.get_pair_table(circuit, circuit.default_population_name)

,nrn1,nrn2,nsyn_ff,nsyn_fb,nsyn_all,is_rc
17,6,8,13,0,13,False


In [10]:
# Neuron set expression
neuron_ids = neuron_set.get_neuron_ids(circuit, circuit.default_population_name)
print(f"{neuron_set.__class__.__name__} resolved in population '{circuit.default_population_name}' of circuit '{circuit}':")
print(f"> Neuron IDs ({len(neuron_ids)}): {neuron_ids}")
print(f"> Node set dict: {neuron_set.get_node_set_definition(circuit, circuit.default_population_name)}")

PairMotifNeuronSet resolved in population 'S1nonbarrel_neurons' of circuit 'N_10__top_nodes_dim6':
> Neuron IDs (2): [6 8]
> Node set dict: {'population': 'S1nonbarrel_neurons', 'node_id': [6, 8]}
